# Разработка компонентов приложения для предсказания уровня дисфункции легких.  
Аршакян Гаянэ Робертовна  
Студентка 4 курса 9 группы  
Фундаментальная информатика и информационные технологии

## 1. Описание и анализ входных данных

In [ ]:
import numpy as np
import pandas as pd
import os

#### Фрагмент датафрейма

In [ ]:
train_data = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/train.csv")
train_data.head()

In [ ]:
print("Количество записей о пациентах : ", train_data.shape[0])
print("Количество пациентов : ", len(train_data.groupby('Patient')))

In [ ]:
print("Минимальное количество записей о пациенте: ", train_data.groupby("Patient")["Weeks"].count().min())
print("Максимальное количество записей о пациенте: ", train_data.groupby("Patient")["Weeks"].count().max())

#### Smoking Status - статус курения

In [ ]:
for value in train_data.SmokingStatus.unique():
    print(value)

In [ ]:
import plotly.express as px
fig = px.sunburst(data_frame=train_data,
                  path=['Age', 'Sex', 'SmokingStatus'],
                  color='Sex',
                  title='Диаграмма SunBurst SmokingStatus x Sex x Age')

fig.update_traces(textinfo='label+percent parent')
fig.show()

In [ ]:
# датафрейм без повторений пациентов
trainDataDropDuplicate = train_data.drop_duplicates(subset='Patient',keep='first') 

In [ ]:
import seaborn as sns
sns.catplot(x="SmokingStatus", y="Age", hue="Sex", kind="swarm", data=trainDataDropDuplicate);

#### Age - возраст

In [ ]:
sns.distplot(train_data['Age'],hist=True)

In [ ]:
from matplotlib import pyplot as plt
ageDf = pd.DataFrame(train_data.Age.value_counts().head(10))
ageDf.reset_index(inplace=True)
ageDf.columns = ['Возраст','Количество записей пациентов']

plt.figure(figsize = (9, 10))
plt.title("10 возрастов с наибольшим количеством записей")
sns.barplot(x="Возраст", y="Количество записей пациентов", data=ageDf,
            label="Count")
plt.show()

#### FVC и Percent

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize = (16, 6))

a = sns.distplot(train_data["FVC"], ax=ax1, hist=True)
b = sns.distplot(train_data["Percent"], ax=ax2, hist=True)

In [ ]:
sns.catplot(x="SmokingStatus", y="Percent", hue="Sex", kind="box", data=trainDataDropDuplicate);

In [ ]:
train_data.corr()

In [ ]:
plt.figure(figsize = (4, 6))
sns.scatterplot(x = train_data["FVC"], y = train_data["Percent"], palette=['#74a09e','#c14953'],
                    hue = train_data["Sex"], style = train_data["Sex"], s=100)


In [ ]:
dataCountWeek = train_data.groupby("Patient")["Weeks"].count().reset_index()
train_data["Time"] = 0

for patient, times in zip(dataCountWeek["Patient"], dataCountWeek["Weeks"]):
    train_data.loc[train_data["Patient"] == patient, 'Time'] = range(1, times+1)
# Вычисляем разницу в FVC
minFVC = train_data[train_data["Time"] == 1]
maxFVC = train_data[train_data["Weeks"] == train_data.groupby(["Patient"])["Weeks"].transform(max)]

data = pd.merge(minFVC, maxFVC, on="Patient")
data["DifFVC"] = data["FVC_x"] - data["FVC_y"]

# Выбираем первые 100 пациентов с большей разницей FVC
lst = list(data.sort_values("DifFVC", ascending=False).head(100)["Patient"])
maxDifFvcPatients = train_data[train_data["Patient"].isin(lst)]

#отрисовка графиков
plt.figure(figsize = (16, 6))
sns.lineplot(x = maxDifFvcPatients["Time"], y = maxDifFvcPatients["FVC"], hue = maxDifFvcPatients["Patient"], legend=False, size=1)
plt.title("Убывание ФЖЕЛ пациента по неделям", fontsize = 16)
plt.xlabel("Weeks", fontsize=14)
plt.ylabel("FVC", fontsize=14);

## 2. Обработка изображений КТ

In [ ]:
import pydicom
from skimage import measure
from skimage import morphology
from sklearn.cluster import KMeans

In [ ]:
path= '../input/osic-pulmonary-fibrosis-progression/train/'
patients = os.listdir(path)
image_counts = []
for p in patients:
    number_of_images= len(os.listdir(path+p))
    image_counts.append(number_of_images)
    
print("Минимальное количество изображений КТ у одного пациента: ", np.min(image_counts))
print("Максимальное количество изображений КТ у одного пациента: ", np.max(image_counts))


##### Следующий метод загрузки изображений КТ взят из книги 
##### Professor Olev Martens, 3D Deep multi-neural networks using serializtion and sigmoid averaging for lung nodule classification. –  Tallinn, 2020

In [ ]:
def load_scan(path):
    slices = [pydicom.dcmread(path + '/' + s) for s in               
              os.listdir(path)]
    slices = [s for s in slices if 'SliceLocation' in s]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [ ]:
first_patient = train_data['Patient'][0]
path1 = '../input/osic-pulmonary-fibrosis-progression/train/' + first_patient
patient_dicom1 = load_scan(path1)
patient_pixels1 = get_pixels_hu(patient_dicom1)
# взят третий скан первого пациента
plt.figure(figsize = (15,5))
plt.imshow(patient_pixels[3],cmap='gray')

In [ ]:
plt.hist(patient_pixels.flatten(), bins=50)
plt.xlabel("Единицы Хаунсфилда (HU)")
plt.show()

In [ ]:
path= '../input/osic-pulmonary-fibrosis-progression/train/'
patients = os.listdir(path)
image_counts = []
for p in patients:
    number_of_images= len(os.listdir(path+p))
    image_counts.append(number_of_images)
    
print("Минимальное количество изображений : ", np.min(image_counts))
print("Максимальное количество изображений : ", np.max(image_counts))


In [ ]:
path='../input/osic-pulmonary-fibrosis-progression/train/ID00019637202178323708467/1.dcm'
image= pydicom.dcmread(path)
print(image)

In [ ]:
path2 ='../input/osic-pulmonary-fibrosis-progression/train/ID00199637202248141386743/1.dcm'
patient = [pydicom.dcmread(path1+"/1.dcm"), pydicom.dcmread(path2)]
print("Толщина среза у первого пациента: ", patient[0].SliceThickness)
print("Физическое расстояние между пикселями у первого пациента: (",patient[0].PixelSpacing[0],", ", patient[0].PixelSpacing[1],")")
print("Толщина среза у второго пациента: ", patient[1].SliceThickness)
print("Физическое расстояние между пикселями у второго пациента: (",patient[1].PixelSpacing[0],", ", patient[1].PixelSpacing[1],")")

**Методы сегментации легких взяты из следующих источников**   
Monica Ramakrishnan, Surya Rajasekaran, Barsa Nayak, Akshay Bhagdikar  Automated lung cancer nodule detection. –  Santa Clara University, 2017.  
Метод водораздела. – URL: https://github.com/Wrosinski/Kaggle-DSB

In [ ]:
from scipy.stats import kurtosis, skew

def get_feautures(array, spacing):
    features = {}
    
    # объем легких
    cube_volume = spacing[0] * spacing[1] * spacing[2]
    total_lung_volume_cube_mm = (array[array > -900].shape[0] * cube_volume)
    features['lung_volume_in_liters'] = total_lung_volume / (1000*1000) # перевод из кубических мм в литры
    #статистические характеристики
    lung_threshold = (array <= -320) & (array >= -900)
    histogram_values, _ = np.histogram(array[lung_threshold].flatten(), bins=100)
    features['lung_mean_hu'] = np.mean(array[lung_threshold].flatten())
    features['lung_skew'] = skew(histogram_values)
    features['lung_kurtosis'] = kurtosis(histogram_values)
    # соотношения единиц Хаунсфилда
    bins_threshold = (array <= 300) & (array >= -900)
    total_hu_units_bin = array[bins_threshold].flatten().shape[0]
    bin_values, bins = np.histogram(array[bins_threshold].flatten(), bins=range(-900, 400, 100))
    features['total_hu_units_bin'] = total_hu_units_bin
    for i, _bin in enumerate(bins[:-1]):
        features[f'bin_{_bin}'] = bin_values[i] / total_hu_units_bin
    # высота легких в см
    n_lung_pixels = lung_threshold.sum(axis=1).sum(axis=1)
    height_start = np.argwhere(n_lung_pixels > 1000).min()
    height_end = np.argwhere(n_lung_pixels > 1000).max()
    features['height_of_lung_cm'] = (height_end - height_start)/10 #высота легких в см
    
    return features

In [ ]:
def get_resampled_array(patient_path):
    start_time = time.time()
    patient_slices = load_scan(str(patient_path))
    patient_slices_hu = get_pixels_hu(patient_slices)
    print('HU loaded', time.time() - start_time)
    idx = np.ndindex(patient_slices_hu.shape[0])
    patient_slices_hu_masked = np.zeros(patient_slices_hu.shape)
    for i in idx:
        patient_slices_hu_masked[i] = seperate_lungs(patient_slices_hu[i]) # метод водораздела(выше ссылка)
    resampled_array, spacing = torch_resample(patient_slices_hu_masked, patient_slices, [1,1,1])
    print('after', time.time() - start_time)
    return resampled_array, spacing

In [ ]:
def get_resampled_array2(patient_path):
    start_time = time.time()
    patient_slices = load_scan(patient_path)
    patient_slices_hu = get_pixels_hu(patient_slices)
    print('HU loaded', time.time() - start_time)
    imgs_after_resamp, spacing = resample2(patient_slices_hu, patient_slices)
    patient_slices_hu_masked = np.zeros(imgs_after_resamp.shape)
    idx = np.ndindex(imgs_after_resamp.shape[0])
    for i in idx:
        patient_slices_hu_masked[i] = make_lungmask(imgs_after_resamp[i])# метод из публикации(выше публикация)
    print('after', time.time() - start_time)
    return patient_slices_hu_masked, spacing

In [ ]:
import traceback
import warnings
import time
from pathlib import Path
warnings.filterwarnings('ignore')

data_dir = Path('../input/osic-pulmonary-fibrosis-progression')
patient_paths = list((data_dir/'train').glob('*'))
patients_feature_df = pd.DataFrame() # комментируемая строка, чтобы запускать для разных диапозонов пациентов
st = time.time()
# Делаем по 25 пациентов, чтобы ресурсов на вычисление хватило
for patient_path in patient_paths:# [0,25],[25,50],[50,75],[75,100],[100,125],[125,150],[150,176]
    try:
        array, spacing = get_resampled_array(str(patient_path))# метод который вовзращает массив после применения маски и ресемплинга
        features = get_features(array, spacing)
        features['patient_id'] = str(patient_path).split('/')[-1]
        features['missing'] = 0
    except Exception as e:
        features = {}
        features['patient_id'] = str(patient_path).split('/')[-1]
        features['missing'] = 1
        print(e)
    patient_df = pd.DataFrame(pd.Series(features)).T
    patients_feature_df = pd.concat([patients_feature_df, patient_df], ignore_index=True)
    patients_feature_df.to_csv('features_df.csv', index=False)

## 3. Обучение модели

In [ ]:
from sklearn import linear_model
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
train_data = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/train.csv")
df = train_data
df_target = pd.DataFrame(df['FVC'])
del df['FVC']

In [ ]:
df = df.drop("Patient", axis=1)
df["Sex"] = df["Sex"].map({'Male': 2, 'Female': 1})
df["SmokingStatus"] = df["SmokingStatus"].map({'Ex-smoker': 3, 'Never smoked': 1, 'Currently smokes': 2})

In [ ]:
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train,y_test = train_test_split(df,df_target,test_size = 0.2,random_state=4)
model = linear_model.LinearRegression()
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model,parameters, cv=5)
grid.fit(X_train, y_train)
grid.best_params_


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
print("Best score : ", grid.best_score_)
print("Mean squared error : ",mean_squared_error(y_test, grid.predict(X_test)))
print("Mean squared log error : ",mean_squared_log_error(y_test, grid.predict(X_test)))
print("Median absolute error : ",median_absolute_error(y_test, grid.predict(X_test)))
print("Mean absolute error : ",mean_absolute_error(y_test, grid.predict(X_test)))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
regressor.fit(X_train, y_train)
print("Score : ", regressor.score(X_test,y_test))
print("Mean squared error : ",mean_squared_error(y_test, regressor.predict(X_test)))
print("Mean squared log error : ",mean_squared_log_error(y_test, regressor.predict(X_test)))
print("Median absolute error : ",median_absolute_error(y_test, regressor.predict(X_test)))
print("Mean absolute error : ",mean_absolute_error(y_test, regressor.predict(X_test)))

In [ ]:
lasso = linear_model.Lasso()
x_train3, x_test3, y_train3,y_test3 = train_test_split(df,df_target,test_size = 0.2,random_state=4)
lasso.fit(x_train3,y_train3)
lasso.coef_
print("Mean squared error : ",mean_squared_error(y_test, lasso.predict(X_test)))
print("Mean squared log error : ",mean_squared_log_error(y_test, lasso.predict(X_test)))
print("Median absolute error : ",median_absolute_error(y_test, lasso.predict(X_test)))
print("Mean absolute error : ",mean_absolute_error(y_test, lasso.predict(X_test)))
print("Количество признаков, коэффициенты Лассо которых не равны 0",sum( (1 if i!=0 else 0) for i in lasso.coef_))

In [ ]:
copy_df = df.copy()
copy_df.drop(['SmokingStatus','lung_volume_in_liters_watershed','bin_-900_watershed',
       'bin_-800_watershed', 'bin_-700_watershed', 'bin_-600_watershed',
       'bin_-500_watershed', 'bin_-400_watershed', 'bin_-300_watershed',
       'bin_-200_watershed', 'bin_-100_watershed', 'bin_0_watershed',
       'bin_100_watershed', 'bin_200_watershed','lung_skew_watershed','bin_-900', 'bin_-800', 'bin_-700', 'bin_-600',
       'bin_-500', 'bin_-400', 'bin_-300', 'bin_-200', 'bin_-100', 'bin_0',
       'bin_100', 'bin_200','lung_skew'], axis='columns', inplace=True)

**Произведения признаков вплоть до 4 множителей**

In [ ]:
n=4
copy_df = df.copy()
length = df.shape[1]
for i in range(1,n):
    l= copy_df.shape[1]
    columns = copy_df.columns
    for m in range(0,length):
        for k in range(m,l):
            s = columns[m] +" x "+ columns[k]
            copy_df[s]= copy_df.iloc[:,m] * copy_df.iloc[:,k]

In [ ]:
lasso = linear_model.Lasso()
x_train3, x_test3, y_train3,y_test3 = train_test_split(copy_df,df_target,test_size = 0.2,random_state=4)
lasso.fit(x_train3,y_train3)
print("Количество признаков, коэффициенты Лассо которых не равны 0 с точностью 0.00000000001",sum( (1 if abs(i)>0.00000000001 else 0) for i in lasso.coef_))

In [ ]:
arr = []
for i in range(len(lasso.coef_)):
    if abs(lasso.coef_[i])<0.00000000001:
       arr.insert(0,copy_df.columns[i])
copy_df.drop(arr, axis='columns', inplace=True)

In [ ]:
X_train1, X_test1, y_train1,y_test1 = train_test_split(z,df_target,test_size = 0.2,random_state=4)
model = linear_model.LinearRegression()
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid3 = GridSearchCV(model,parameters, cv=5)
grid3.fit(X_train1, y_train1)
grid3.best_params_
print("Best score : ", grid3.best_score_)
print("Mean squared error : ",mean_squared_error(y_test1, grid3.predict(X_test1)))
print("Mean squared log error : ",mean_squared_log_error(y_test1, grid3.predict(X_test1)))
print("Median absolute error : ",median_absolute_error(y_test1, grid3.predict(X_test1)))
print("Mean absolute error : ",mean_absolute_error(y_test1, grid3.predict(X_test1)))

In [ ]:
parameters = {'criterion':['mse', 'friedman_mse', 'mae'],'min_samples_leaf':range(1,5),'min_samples_split':range(1,5)}
grid4 = GridSearchCV(regressor,parameters, cv=5)
grid4.fit(X_train1, y_train1)
grid4.best_params_

In [ ]:
first = regressor.predict(X_test1) * 0.8
first += grid4.predict(X_test1) * 0.2
print("Mean squared error : ",mean_squared_error(y_test1, first))
print("Mean squared log error : ",mean_squared_log_error(y_test1, first))
print("Median absolute error : ",median_absolute_error(y_test1, first))
print("Mean absolute error : ",mean_absolute_error(y_test1, first))

In [ ]:
second = regressor.predict(X_test1) *0.5
second += grid4.predict(X_test1) *0.5
print("Mean squared error : ",mean_squared_error(y_test1, second))
print("Mean squared log error : ",mean_squared_log_error(y_test1, second))
print("Median absolute error : ",median_absolute_error(y_test1, second))
print("Mean absolute error : ",mean_absolute_error(y_test1, second))

In [ ]:
third = regressor.predict(X_test1) *0.65
third += grid4.predict(X_test1) *0.35
print("Mean squared error : ",mean_squared_error(y_test1, third))
print("Mean squared log error : ",mean_squared_log_error(y_test1, third))
print("Median absolute error : ",median_absolute_error(y_test1, third))
print("Mean absolute error : ",mean_absolute_error(y_test1, third))

In [ ]:
forth = regressor.predict(X_test1) *0.2
forth += grid4.predict(X_test1) *0.8
print("Mean squared error : ",mean_squared_error(y_test1, forth))
print("Mean squared log error : ",mean_squared_log_error(y_test1, forth))
print("Median absolute error : ",median_absolute_error(y_test1, forth))
print("Mean absolute error : ",mean_absolute_error(y_test1, forth))